In [ ]:
# Install and setup Kaggle
!pip install -q kaggle
from google.colab import files
files.upload()  # Upload your kaggle.json
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
# Download and extract dataset (correct path)
!kaggle datasets download -d gpiosenka/musical-instruments-image-classification
!unzip -q musical-instruments-image-classification.zip


Dataset URL: https://www.kaggle.com/datasets/gpiosenka/musical-instruments-image-classification
License(s): CC0-1.0
 97% 198M/204M [00:00<00:00, 617MB/s]  
100% 204M/204M [00:00<00:00, 632MB/s]


In [ ]:
# Correct paths (verified)
train_dir = 'train'
test_dir = 'test'
img_size = (299, 299)
batch_size = 32

In [ ]:
# Verify files
!ls train
!ls test

import tensorflow as tf
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

 acordian      casaba	    Didgeridoo	 guitar      piano	   trombone
 alphorn       castanets    drums	 harmonica   saxaphone	   trumpet
 bagpipes      clarinet     dulcimer	 harp	     sitar	   tuba
 banjo	       clavichord   flute	 marakas    'steel drum'   violin
'bongo drum'   concertina   guiro	 ocarina     Tambourine    Xylophone
 acordian      casaba	    Didgeridoo	 guitar      piano	   trombone
 alphorn       castanets    drums	 harmonica   saxaphone	   trumpet
 bagpipes      clarinet     dulcimer	 harp	     sitar	   tuba
 banjo	       clavichord   flute	 marakas    'steel drum'   violin
'bongo drum'   concertina   guiro	 ocarina     Tambourine    Xylophone


In [ ]:
# Data generators
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 4793 images belonging to 30 classes.
Found 150 images belonging to 30 classes.


In [ ]:
# Build model
base_model = InceptionResNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(*img_size, 3)
)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:

# Train
history = model.fit(
    train_generator,
    epochs=3,
    validation_data=test_generator
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/3
150/150 ━━━━━━━━━━━━━━━━━━━━ 409s 2s/step - accuracy: 0.7122 - loss: 1.3686 - val_accuracy: 0.9800 - val_loss: 0.1178
Epoch 2/3
150/150 ━━━━━━━━━━━━━━━━━━━━ 155s 1s/step - accuracy: 0.9740 - loss: 0.1051 - val_accuracy: 0.9933 - val_loss: 0.0139
Epoch 3/3
150/150 ━━━━━━━━━━━━━━━━━━━━ 155s 1s/step - accuracy: 0.9882 - loss: 0.0430 - val_accuracy: 1.0000 - val_loss: 0.0105


In [ ]:
# Evaluate
loss, accuracy = model.evaluate(test_generator)
print(f"\nTest Accuracy: {accuracy * 100:.2f}%")

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 246ms/step - accuracy: 1.0000 - loss: 0.0104

Test Accuracy: 100.00%


In [ ]:
# Save
model.save("instrument_classifier.h5")
print("Model saved successfully!")

Model saved successfully!
